**Task 5**


1.   Finding out most discussed 10 topics using LDA

Aditional Steps

1.   Finding out the performance of the generated LDA
2.   FInding out best optimal number of topics for a better model
3.   Analysis using pyLadvis


Inputs for this task are files from the location data/Task 5/input
These files are same as the files in the data/Task 1/Result (output from the task 1)








In [1]:
import spacy
import nltk

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'use'])
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sasini\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from tabulate import tabulate
import pyLDAvis
import pyLDAvis.gensim_models as genismnvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

topicsname=[]
num_topics =10
bigramDataWords = []
trigramDataWords = []
EvaluationResults =[]
appDetailInputsForLDA =[]
ldaModelDetailsTfAppWise =[]
ldaModelDetailsTfIdfAppWise =[]

#function for preprocessing Data
def PreProcessReviewData(reviewData):
    for review in reviewData:
        yield gensim.utils.simple_preprocess(str(review), deacc=True, min_len=2)  # deacc=True removes punctuations

#function for removing stopwords
def removeStopWords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

#function for bigram creation 
def make_bigrams(texts):
    for doc in texts:
        x = bigram_mod[doc]
        bigramDataWords.append(x)
    return bigramDataWords

#function for trigram creation 
def make_trigrams(texts):
    for doc in texts:
        y = trigram_mod[bigram_mod[doc]]
        trigramDataWords.append(y)
    return trigramDataWords

#function for lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#function for training LDA model with Tf corpus
def TrainLdaModelWithTf(corpus, id2word, topics = 10, passes= 10, iterations =50) :
    ldaModel = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=topics,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=passes,
                                            alpha='auto',
                                            per_word_topics=True,
                                            iterations=iterations)
    return ldaModel

#function for training LDA model with TfIdf corpus
def TrainLdaModelWithTfIdf(corpus, id2word, topics = 10, passes= 10, iterations =50) :
    ldaModel = gensim.models.LdaMulticore(corpus=corpus,
                                            id2word=id2word,
                                            num_topics= topics,
                                            random_state=100,
                                            chunksize=100,
                                            passes=passes,
                                            per_word_topics=True,
                                            iterations= iterations)
    return ldaModel

#function for evaluate LDA model
def EvaluatePerformance(appName, corpus, ldaModel, corpusType):
    results =[appName, corpusType]
    perplexity = ldaModel.log_perplexity(corpus)

    # Compute CV Coherence Score
    cvCoherenceModel = CoherenceModel(model=ldaModel, texts=lemmatizedData, dictionary=id2word, coherence='c_v')
    cvCoherence = cvCoherenceModel.get_coherence()

    # Compute umass Coherence Score
    uMassCoherenceModel = CoherenceModel(model=ldaModel, texts=lemmatizedData, dictionary=id2word, coherence='u_mass')
    uMassCoherence = uMassCoherenceModel.get_coherence()

    results.append(perplexity)
    results.append(cvCoherence)
    results.append(uMassCoherence)

    return results

#function for printing words in the topic
def PrintTopicWords(wordSet):
    topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in wordSet]
    for topic,words in topics_words:
        print(str(topic)+ "::"+ str(words))

#function for plpting LDA model generated topics
def PlottingResults(ldamodel, corpus, id2word):
    pyLDAvis.enable_notebook()
    vis = genismnvis.prepare(ldamodel, corpus, id2word)
    return vis


apps = ['Emergency Plus.csv', 'Red Cross First Aid.csv']

for i in range(len(apps)) :
    df = pd.read_csv('./data/Task 5/Input/'+apps[i])
    reviewDataList = df['Review'].values.tolist()

    #preprocessing using genism simple pre process
    reviewDataWords = list(PreProcessReviewData(reviewDataList))

    #stopword removal using nltk stop words
    reviewDataWordsWithoutStopWords = removeStopWords(reviewDataWords)

    # bigram trigram models
    bigram = gensim.models.Phrases(reviewDataWords, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[reviewDataWords], threshold=100)

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
  
    #creating bigram trigram models from the data set
    dataWordsWithbigrams = make_bigrams(reviewDataWordsWithoutStopWords)
    dataWordsWithTrigrams = make_trigrams(dataWordsWithbigrams)

    #lemmatizing data set
    lemmatizedData = lemmatization(dataWordsWithTrigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

  
    #create dictionary as the input for lda model
    id2word = corpora.Dictionary(lemmatizedData)

    # Create Corpus
    texts = lemmatizedData

    # create corpus with Term Document Frequency
    corpusTf = [id2word.doc2bow(text) for text in texts]

    #create corpus with tf-idf
    tfidf = gensim.models.TfidfModel(corpusTf)
    corpusTfidf = tfidf[corpusTf]
  
    #store details in dictionary for use later for evaluation
    dictionary = {'appName':apps[i],'id2word':id2word, 'lemmatized': lemmatizedData, 'tf' : corpusTf, 'tfIdf': corpusTfidf }
    appDetailInputsForLDA.append(dictionary)

    ldaModelTfDetails =[]
    ldaModelTfIdfDetails =[]

    ldaModelTf = TrainLdaModelWithTf(corpusTf, id2word)
    ldaModelTfWords = ldaModelTf.show_topics(num_topics=10, num_words=10,formatted=False)
    print("\n 10 Topics obtained from the reviews of "+apps[i]+ " - Tf Corpus")
    PrintTopicWords(ldaModelTfWords)
    #pprint(ldaModelTf.print_topics()) -print with probabilities
    results = EvaluatePerformance(apps[i], corpusTf, ldaModelTf, corpusType = 'TF')
    EvaluationResults.append(results)

    #Store details to utilize when plotting graphs
    ldaModelTfDetails.append(ldaModelTf)
    ldaModelTfDetails.append(corpusTf)
    ldaModelTfDetails.append(id2word)
    ldaModelDetailsTfAppWise.append(ldaModelTfDetails)

    ldaModelTfIdf = TrainLdaModelWithTfIdf(corpusTfidf, id2word)
    ldaModelTIdffWords = ldaModelTfIdf.show_topics(num_topics=10, num_words=10,formatted=False)
    print("\n 10 Topics obtained from the reviews of "+apps[i]+ " - TfIdf Corpus")
    PrintTopicWords(ldaModelTIdffWords)
    #pprint(ldaModelTfIdf.print_topics()) -print with probabilities
    results = EvaluatePerformance(apps[i], corpusTfidf, ldaModelTfIdf, corpusType = 'TF-IDF')
    EvaluationResults.append(results)
  
    #Store details to utilize when plotting graphs
    ldaModelTfIdfDetails.append(ldaModelTfIdf)
    ldaModelTfIdfDetails.append(corpusTfidf)
    ldaModelTfIdfDetails.append(id2word)
    ldaModelDetailsTfIdfAppWise.append(ldaModelTfIdfDetails)

print (tabulate(EvaluationResults, headers=["App Name", "Corpus Type" "Perplexity", "Cv coherence score", "U mass Coherence score"]))

C:\Users\Sasini\AppData\Local\Programs\Python\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload



 10 Topics obtained from the reviews of Emergency Plus.csv - Tf Corpus
0::['life', 'save', 'much', 'locate', 'accident', 'apple_watch', 'able', 'review', 'test', 'quickly']
1::['app', 'emergency', 'location', 'great', 'need', 'address', 'phone', 'gps', 'good', 'idea']
2::['try', 'away', 'tell', 'area', 'however', 'correct', 'rural', 'name', 'live', 'family']
3::['people', 'button', 'contact', 'map', 'share', 'mobile', 'alert', 'helpful', 'direct', 'maybe']
4::['want', 'mean', 'far', 'kilometre', 'logo', 'perfectly', 'new', 'recent', 'hope_never', 'clear']
5::['long', 'keep', 'actually', 'take', 'go', 'report', 'exact', 'small', 'old', 'seem']
6::['also', 'read', 'make', 'see', 'download', 'screen', 'useful', 'well', 'information', 'turn']
7::['work', 'update', 'fix', 'still', 'iphone', 'app', 'fire', 'phone', 'application', 'handy']
8::['know', 'recommend', 'situation', 'dispatch', 'always', 'life_save', 'bad', 'due', 'often', 'highly']
9::['time', 'home', 'open', 'crash', 'operator',

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True,
                                            iterations=50)
                                            
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
def PlotCoherenceValueAgainstNumofTopics(appName, model, coherence_values):
    limit=40; start=2; step=6;
    x = range(start, limit, step)
    plt.plot(x, coherence_values)
    plt.title(appName)
    plt.xlabel("Num Topics")
    plt.ylabel("Coherence score "+ model)
    plt.legend(("coherence_values"), loc='best')
    plt.show()


In [ ]:
#appDetailInputsForLDA containing details related to emergency app
model_list, coherence_values = compute_coherence_values(dictionary=appDetailInputsForLDA[0]['id2word'], corpus=appDetailInputsForLDA[0]['tf'], texts=appDetailInputsForLDA[0]['lemmatized'], start=2, limit=40, step=6)
PlotCoherenceValueAgainstNumofTopics("Emergency Plus app", "tf model", coherence_values)

model_list, coherence_values = compute_coherence_values(dictionary=appDetailInputsForLDA[0]['id2word'], corpus=appDetailInputsForLDA[0]['tfIdf'], texts=appDetailInputsForLDA[0]['lemmatized'], start=2, limit=40, step=6)
PlotCoherenceValueAgainstNumofTopics("Emergency Plus app", "tf-idf model", coherence_values)

In [ ]:
#appDetailInputsForLDA containing details related to red cross first aid app
model_list, coherence_values = compute_coherence_values(dictionary=appDetailInputsForLDA[1]['id2word'], corpus=appDetailInputsForLDA[1]['tf'], texts=appDetailInputsForLDA[1]['lemmatized'], start=2, limit=40, step=6)
PlotCoherenceValueAgainstNumofTopics("Red cross first aid app", "tf model", coherence_values)

model_list, coherence_values = compute_coherence_values(dictionary=appDetailInputsForLDA[1]['id2word'], corpus=appDetailInputsForLDA[1]['tfIdf'], texts=appDetailInputsForLDA[1]['lemmatized'], start=2, limit=40, step=6)
PlotCoherenceValueAgainstNumofTopics("ed cross first aid app", "tf-idf model", coherence_values)

In [ ]:
# same process is rerunning just changing the num of topics to optimal number of topics.

bigramDataWords = []
trigramDataWords = []
EvaluationResults =[]
appDetailInputsForLDA =[]
ldaModelDetailsTfAppWise =[]
ldaModelDetailsTfIdfAppWise =[]


apps = ['Emergency Plus.csv', 'Red Cross First Aid.csv']

for i in range(len(apps)) :
    if i== 0 :
        numberOfTfTopics = 13
        numberofTfIdfTopics = 8
    else:
        numberOfTfTopics = 14
        numberofTfIdfTopics = 8

    df = pd.read_csv('/content/'+apps[i])
    reviewDataList = df['Review'].values.tolist()

    #preprocessing using genism simple pre process
    reviewDataWords = list(PreProcessReviewData(reviewDataList))

    #stopword removal using nltk stop words
    reviewDataWordsWithoutStopWords = removeStopWords(reviewDataWords)

    # bigram trigram models
    bigram = gensim.models.Phrases(reviewDataWords, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[reviewDataWords], threshold=100)

    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
  
    #creating bigram trigram models from the data set
    dataWordsWithbigrams = make_bigrams(reviewDataWordsWithoutStopWords)
    dataWordsWithTrigrams = make_trigrams(dataWordsWithbigrams)

    #lemmatizing data set
    lemmatizedData = lemmatization(dataWordsWithTrigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

  
    #create dictionary as the input for lda model
    id2word = corpora.Dictionary(lemmatizedData)

    # Create Corpus
    texts = lemmatizedData

    # create corpus with Term Document Frequency
    corpusTf = [id2word.doc2bow(text) for text in texts]

    #create corpus with tf-idf
    tfidf = gensim.models.TfidfModel(corpusTf)
    corpusTfidf = tfidf[corpusTf]
  
    #store details in dictionary for use later for evaluation
    dictionary = {'appName':apps[i],'id2word':id2word, 'lemmatized': lemmatizedData, 'tf' : corpusTf, 'tfIdf': corpusTfidf }
    appDetailInputsForLDA.append(dictionary)

    ldaModelTfDetails =[]
    ldaModelTfIdfDetails =[]

    ldaModelTf = TrainLdaModelWithTf(corpusTf, id2word, numberOfTfTopics)
    ldaModelTfWords = ldaModelTf.show_topics(num_topics=10, num_words=10,formatted=False)
    print("\n ")
    print(numberOfTfTopics)
    print(" Topics obtained from the reviews of "+apps[i]+ " - Tf Corpus")
    PrintTopicWords(ldaModelTfWords)
    #pprint(ldaModelTf.print_topics()) -print with probabilities
    results = EvaluatePerformance(apps[i], corpusTf, ldaModelTf, corpusType = 'TF')
    EvaluationResults.append(results)

    #Store details to utilize when plotting graphs
    ldaModelTfDetails.append(ldaModelTf)
    ldaModelTfDetails.append(corpusTf)
    ldaModelTfDetails.append(id2word)
    ldaModelDetailsTfAppWise.append(ldaModelTfDetails)

    ldaModelTfIdf = TrainLdaModelWithTfIdf(corpusTfidf, id2word, numberofTfIdfTopics)
    ldaModelTIdffWords = ldaModelTfIdf.show_topics(num_topics=10, num_words=10,formatted=False)
    print("\n ")
    print(numberofTfIdfTopics)
    print(" Topics obtained from the reviews of "+apps[i]+ " - TfIdf Corpus")
    PrintTopicWords(ldaModelTIdffWords)
    #pprint(ldaModelTfIdf.print_topics()) -print with probabilities
    results = EvaluatePerformance(apps[i], corpusTfidf, ldaModelTfIdf, corpusType = 'TF-IDF')
    EvaluationResults.append(results)
  
    #Store details to utilize when plotting graphs
    ldaModelTfIdfDetails.append(ldaModelTfIdf)
    ldaModelTfIdfDetails.append(corpusTfidf)
    ldaModelTfIdfDetails.append(id2word)
    ldaModelDetailsTfIdfAppWise.append(ldaModelTfIdfDetails)

In [ ]:
#plotting for emergency app tf corpus
vis =PlottingResults(ldaModelDetailsTfAppWise[0][0], ldaModelDetailsTfAppWise[0][1], ldaModelDetailsTfAppWise[0][2])
vis

In [ ]:
#plotting for emergency app tf idf corpus
vis =PlottingResults(ldaModelDetailsTfIdfAppWise[0][0], ldaModelDetailsTfIdfAppWise[0][1], ldaModelDetailsTfIdfAppWise[0][2])
vis

In [ ]:
#plotting for red cross app tf corpus
vis =PlottingResults(ldaModelDetailsTfAppWise[1][0], ldaModelDetailsTfAppWise[1][1], ldaModelDetailsTfAppWise[1][2])
vis

In [ ]:
#plotting for red cross first aid app tf idf corpus
vis =PlottingResults(ldaModelDetailsTfIdfAppWise[1][0], ldaModelDetailsTfIdfAppWise[1][1], ldaModelDetailsTfIdfAppWise[1][2])
vis